In [1]:
import pandas as pd
import pandera as pa

In [2]:
valores_ausentes = ['*****','****','####','###!','**','NULL','***']
df = pd.read_csv('ocorrencia.csv', parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes) #fazer a leitura da tablea, parse_data: faz a conversão para data e dayfirst: faz com que o dia seja o primeiro número
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [3]:
schema = pa.DataFrameSchema( #cria um esquema para o data frame para fazer as validações
    columns = {
    'codigo':pa.Column(pa.Int, required=False), # O valor required (necessário) é True por padrao, colocando assim essa passa a ser uma coluna opcional, em caso de ausência não havera erro.
    'codigo_ocorrencia':pa.Column(pa.Int),# diz que a primeira coluna tem que ser int
    'codigo_ocorrencia2':pa.Column(pa.Int), # e assim por diante, verificando o tipo de cada coluna
    'ocorrencia_classificacao':pa.Column(pa.String),
    'ocorrencia_cidade':pa.Column(pa.String),
    'ocorrencia_uf':pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True), # verifica o tamanho min e max
    'ocorrencia_aerodromo':pa.Column(pa.String, nullable=True),
    'ocorrencia_dia':pa.Column(pa.DateTime),
    'ocorrencia_hora':pa.Column(pa.String, pa.Check.str_matches(r'([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'),nullable=True), #nullable é pra permitir valores nulos
    'total_recomendacoes':pa.Column(pa.Int) #essa expressão recular acima faz a validação das horas, minutos e segundos
    }
)

In [4]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6087,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6088,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6089,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6090,80257,80257,ACIDENTE,DOM AQUINO,MT,NaN,2021-08-18,16:00:00,0


In [5]:
# Posso também mesclar colunas

df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

# assim eu consigo concatenar as duas colunas separados por um espaço ''

# usando o método to_datetime eu converto as strings para o formato dia hora 

In [6]:
#ocorrências do ano de 2015 e mês 3
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 3)
df201503 = df.loc[filtro]

In [7]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()
#assim eu conto as classificações mediante ao codigo_ocorrencia, ou seja pra cada ocorrencia, quantas classificações diferentes eu tenho

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [8]:
#Teste aleatório de filtro

filtro = (df.ocorrencia_dia.dt.month == 3) & (df.total_recomendacoes > 0)
df.loc[filtro].groupby(['ocorrencia_uf'], dropna=False).total_recomendacoes.sum().sort_values(ascending=False)

ocorrencia_uf
SP    25
GO    16
PA     9
AM     8
BA     7
PR     7
RJ     7
RS     6
AC     6
MT     4
PI     4
RR     4
ES     3
MS     3
SC     3
MA     2
RO     2
MG     1
Name: total_recomendacoes, dtype: int64